In [1]:
import numpy as np
import pandas as pd
from itertools import combinations

In [2]:
data = pd.read_csv('train_data.csv') 
symbols = 'ABCDEFGHIJ'
base_features = ['open', 'high', 'low', 'close', 'average']

## Data Cleaning

In [3]:
# see if rows are consistent
data_sub = data[data['time'] >= '06:30:00']
for s in symbols:
    d = data_sub[data_sub['symbol'] == s]
    for i in range(87):
        row_ct = d[d.day == i].shape[0]
        if row_ct != 4680:
            print(f"Inconsistent: {row_ct}, i: {i}, s: {s}!")

Inconsistent: 4665, i: 67, s: A!
Inconsistent: 4668, i: 11, s: B!
Inconsistent: 4679, i: 22, s: B!
Inconsistent: 4679, i: 45, s: B!
Inconsistent: 4667, i: 47, s: B!
Inconsistent: 4670, i: 52, s: B!
Inconsistent: 4320, i: 0, s: D!
Inconsistent: 3960, i: 31, s: D!
Inconsistent: 4669, i: 5, s: F!
Inconsistent: 3960, i: 23, s: F!
Inconsistent: 4678, i: 55, s: F!
Inconsistent: 4669, i: 66, s: F!
Inconsistent: 4320, i: 0, s: H!
Inconsistent: 3960, i: 23, s: I!
Inconsistent: 3960, i: 42, s: I!
Inconsistent: 3960, i: 23, s: J!


In [4]:
# make sure dimensions match
full_time = list(data[(data['symbol'] == 'A') & (data['day'] == 2)]['time'])

In [5]:
full_days = [i for i in range(87) for _ in range(len(full_time))]

In [6]:
full_data = pd.DataFrame({"time": full_time*87, "day": full_days})

### Impute missing data based on immediate next non-null value

In [7]:
full_ds = []
for s in symbols:
    d = data[data['symbol'] == s]
    d_full = full_data.merge(d, how = 'left', on = ['time', 'day'])
    d_full.symbol = s
    d_full.fillna(method='bfill', inplace=True)
    full_ds.append(d_full)

In [8]:
data = pd.concat(full_ds)
data.shape

(4384800, 8)

Now it is perfectly 87 days * 5040 timestamps * 10 symbols = 4384800 rows

In [9]:
a = data[data['symbol'] == 'A'].copy()
b = data[data['symbol'] == 'B'].copy()
c = data[data['symbol'] == 'C'].copy()
d = data[data['symbol'] == 'D'].copy()
e = data[data['symbol'] == 'E'].copy()
f = data[data['symbol'] == 'F'].copy()
g = data[data['symbol'] == 'G'].copy()
h = data[data['symbol'] == 'H'].copy()
i = data[data['symbol'] == 'I'].copy()
j = data[data['symbol'] == 'J'].copy()
all_data = list([a, b, c, d, e, f, g, h, i, j])

### Build features

In [10]:
# features 
def get_change(l, s):
    return l / l.shift(s) - 1

get_change(data.open, 1)

0              NaN
1         0.000000
2         0.000000
3         0.000000
4         0.000000
            ...   
438475   -0.000054
438476    0.000000
438477    0.000161
438478    0.000107
438479    0.000161
Name: open, Length: 4384800, dtype: float64

In [11]:
data.open.std()

43.16239080389173

In [12]:
# percentage change over 5s, 1m, 15m, 1h, 1-9d
for d in all_data:
    for bf in base_features:
        d[f'{bf}_5s'] = get_change(d[bf], 1)
        d[f'{bf}_1m'] = get_change(d[bf], 12)
        d[f'{bf}_15m'] = get_change(d[bf], 12*15)
        d[f'{bf}_1h'] = get_change(d[bf], 12*15*4)
        for n in range(1, 10):
            d[f'{bf}_{n}d'] = get_change(d[bf], 12*15*4*7*n)


In [14]:
all_data[0]

,time,day,symbol,open,high,low,close,average,open_5s,open_1m,...,average_1h,average_1d,average_2d,average_3d,average_4d,average_5d,average_6d,average_7d,average_8d,average_9d
0,06:00:00,0,A,135.54,135.79,135.54,135.79,135.67,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,06:00:05,0,A,135.54,135.79,135.54,135.79,135.67,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,06:00:10,0,A,135.54,135.79,135.54,135.79,135.67,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,06:00:15,0,A,135.54,135.79,135.54,135.79,135.67,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,06:00:20,0,A,135.54,135.79,135.54,135.79,135.67,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438475,12:59:35,86,A,146.30,146.32,146.28,146.28,146.29,0.000000,-0.000137,...,0.002673,0.004256,0.006813,-0.013287,-0.015942,-0.013421,-0.031833,-0.026097,-0.021668,-0.005844
438476,12:59:40,86,A,146.28,146.30,146.27,146.29,146.29,-0.000137,-0.000137,...,0.002673,0.004187,0.007438,-0.013287,-0.016075,-0.013421,-0.031961,-0.026226,-0.021733,-0.005912
438477,12:59:45,86,A,146.30,146.32,146.30,146.32,146.31,0.000137,-0.000137,...,0.002810,0.004393,0.007298,-0.013286,-0.016006,-0.013286,-0.031893,-0.025964,-0.021534,-0.005844
438478,12:59:50,86,A,146.29,146.30,146.26,146.26,146.28,-0.000068,-0.000205,...,0.002536,0.004119,0.006953,-0.013421,-0.016407,-0.013288,-0.032028,-0.026358,-0.021735,-0.005912


In [ ]:
# get volatilities of past 1-9d
for d in all_data:
    stds_by_day = d.groupby('day')['open'].std()

    def get_std_by_day(day):
        if day < 0:
            return np.nan
        return stds_by_day[day]

    for n in range(1, 10):
        d[f'vol_{n}d'] = (d['day'] - n).apply(get_std_by_day)


In [ ]:
all_data[0]

In [ ]:
all_data_combined = pd.concat(all_data)

In [ ]:
all_data_combined.to_csv("train_data_transformed.csv")

## Model training

In [80]:
# TODO : train models based on data with added features

def split_train_test(dat):
    return dat[dat['day'] < 78], dat[dat['day'] >= 78]


#### Thoughts: 

##### Training:
- train 9 models, with the ith model to predict i days ahead for each timestamp

Example to train model 1: shift response variable (open price) by 1 day, fit random forest on shifted open price ~ other 80 features

##### Final prediction:
- final prediction will be an ensemble (i.e. exponential weighted average) of the 9 models

Example 1: day 87 at time 7:00:00 will be:
0.5x(model 1 using features at day 86 7:00:00) + 0.25x(model 2 using features at day 85 7:00:00) + 0.125x(model 3 using features at day 84 7:00:00) + ... 

Example 2: day 93 at time 7:00:00 will be:
model 9 using features at day 86 7:00:00
